# 유튜브 크롤링 하기


>  ## 1. 유튜브 API 호출하기


[유튜브크롤링 - 참고자료 - API호출하기](https://blog.naver.com/doublet7411/221511344483)


- (1단계) GCP (Google Cloud Platform) 접속
    - https://console.developers.google.com
    
- (2단계) API 라이브러리에서 "Youtube Data API v3" 선택
    - 사용 클릭
    - 좌측 열쇄모양 아이콘(사용자 인증정보) 클릭
    - 상단 `+사용자 인증정보 만들기` 버튼 클릭 -> `API 키` 클릭
    - API키 복사하기


- (3단계) 구글 포스트맨을 활용하여 정상적으로 호출되는지 확인
    - 구글포스트맨: 크롬 웹스토어에서 `Tabbed Postman` 설치
    - 크롬 확장프로그램 `Tabbed Postman`을 실행.
    - url:  `https://www.googleapis.com/youtube/v3/search`
    - 요청방식: `GET`
    - `URL params 버튼` 클릭
    - URL parameter: `key`
    - Value: `(2단계)에서 발급받은 API키`
 

- postman 사용 결과

![](./results/using_postman.JPG)

> ## 2. 조건에 맞는 유튜브 JSON형식 데이터 조회

- [유튜브 크롤링 참고자료 - 유튜브 크롤링2](https://blog.naver.com/doublet7411/221514043955)

- Tabbed postman

    - (URL Parameter Key) `key`  => (Value) `유튜브 api에서 발급받은 키`
    - (URL Parameter Key) `part`  => (Value) `snippet`
    - (URL Parameter Key) `order`  => (Value) `date`
    - (URL Parameter Key) `q`  => (Value) `찾고자하는 유튜브 이름 (쿼리)`
    
<br>
    
- 파이썬 패키지 설치
    - apiclient
    ```
    pip install apiclient
    ```

    - oauth2client
    ```
    pip install oauth2client
    ```

In [1]:
!pip install apiclient

In [2]:
!pip install oauth2client

In [3]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in c:\users\user\anaconda3\lib\site-packages (1.9.3)


In [6]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser


def youtube_search(query):
    DEVELOPER_KEY= "발급받은 유튜브 API키"
    YOUTUBE_API_SERVICE_NAME="youtube"
    YOUTUBE_API_VERSION="v3"
    
    print('검색결과 => ', query)
    
    # 크롤링 객체
    youtube= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, 
                   developerKey= DEVELOPER_KEY)
    
    # 검색결과 크롤링
    # 검색리스트를 부른다.
    # 쿼리에 명시된것과 일치한 결과물을 메소드에게 반환.
    search_response= youtube.search().list(
        q=query,
        part="snippet",
        maxResults=50
    ).execute()
    
    
    return search_response

In [7]:
youtube_search('강남역 맛집')

검색결과 =>  강남역 맛집


{'kind': 'youtube#searchListResponse',
 'etag': 'tS_GinJGyYFCg1dzxsekQAGJIJk',
 'nextPageToken': 'CDIQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 296551, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'Z9iwMHAYVFXUFAPUBrsvz0xvfSU',
   'id': {'kind': 'youtube#video', 'videoId': '1tD2I9XO3-s'},
   'snippet': {'publishedAt': '2020-03-27T05:47:28Z',
    'channelId': 'UC4h1I4jNS4u-4WNT08mKKqw',
    'title': '[삼카슐랭] #10 서울 강남맛집 종합편 (with 쓰복만)',
    'description': '삼성카드 #삼카슐랭 #쓰복만.',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/1tD2I9XO3-s/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/1tD2I9XO3-s/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/1tD2I9XO3-s/hqdefault.jpg',
      'width': 480,
      'height': 360}},
    'channelTitle': '삼성카드 Samsung Card',
    'liveBroadcastContent': 'none',
    'publishTime': '2020-03-27T05:47

<br>

> ## 내가 입력한 키워드 대한 유튜브 채널 검색

In [8]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser


def youtube_search(query):
    print('검색결과 => ', query)
    DEVELOPER_KEY= "발급받은 유튜브 API키"
    YOUTUBE_API_SERVICE_NAME="youtube"
    YOUTUBE_API_VERSION="v3"
    
    # 크롤링 객체
    youtube= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, 
                   developerKey= DEVELOPER_KEY)
    
    # 검색결과 크롤링
    # 검색리스트를 부른다.
    # 쿼리에 명시된것과 일치한 결과물을 메소드에게 반환.
    search_response= youtube.search().list(
        q=query,
        part="snippet",
        maxResults=50
    ).execute()
    
    answer=[]
    for search_result in search_response.get('items',[]):
        if search_result['id']['kind']=='youtube#video':
            video_id= search_result['id']['videoId']
            title=search_result['snippet']['title']
            description=search_result['snippet']['description']
            thumbnail=search_result['snippet']['thumbnails']['default']['url']
            
            data={
                'url': 'https://youtube.com/watch?v='+video_id,
                'title': title,
                'description': description,
                'photo': thumbnail 
            }
            
            answer.append(data)
            
    return answer
    

In [9]:
answer=youtube_search('강남역 맛집')
answer

검색결과 =>  강남역 맛집


[{'url': 'https://youtube.com/watch?v=1tD2I9XO3-s',
  'title': '[삼카슐랭] #10 서울 강남맛집 종합편 (with 쓰복만)',
  'description': '삼성카드 #삼카슐랭 #쓰복만.',
  'photo': 'https://i.ytimg.com/vi/1tD2I9XO3-s/default.jpg'},
 {'url': 'https://youtube.com/watch?v=W_NgtlZ1m0A',
  'title': '강남 맛집 여기다 있음',
  'description': '강남맛집 #맛집 #먹방 #mukbang #맛집추천 #박가린 #가린 #일상 #브이로그 광고&스폰문의 : business@dreamplug.kr 아프리카TV : http://afreeca.com/janjju ...',
  'photo': 'https://i.ytimg.com/vi/W_NgtlZ1m0A/default.jpg'},
 {'url': 'https://youtube.com/watch?v=eEjn6wktrc4',
  'title': '[맛집 소개] 강남역 맛집 TOP 10',
  'description': '[맛집 소개] 강남역 맛집 TOP 10.',
  'photo': 'https://i.ytimg.com/vi/eEjn6wktrc4/default.jpg'},
 {'url': 'https://youtube.com/watch?v=kP6uWsZedms',
  'title': '❤무한도전이 극찬한 강남의 맛있는집!!! 어육장 ㅣ강남역,역삼역 6분거리ㅣ강남맛집ㅣ하얀트리',
  'description': '무한도전이 종영을 했어요.... ㅡㅠ 아쉬운 마음에 이곳저곳 검색하다 무한도전 멤버들이 개인적으로 찾아오는 맛집이라 해서 왔습니다! 강남역...',
  'photo': 'https://i.ytimg.com/vi/kP6uWsZedms/default.jpg'},
 {'url': 'https://youtube.com/watch?v=7ZnfDdaYps0

In [10]:
len(answer)

50

<BR>


> # 참고블로그 따라해보기


In [11]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

In [12]:
# DEVELOPER_KEY= "(2단계)에서 발급받은 유튜브 API"
# YOUTUBE_API_SERVICE_NAME="youtube"
# YOUTUBE_API_VERSION="v3"


DEVELOPER_KEY= "발급받은 유튜브 API키"
YOUTUBE_API_SERVICE_NAME="youtube"
YOUTUBE_API_VERSION="v3"

def youtube_search(query):
    print('검색결과 => ', query)
    
    # 크롤링 객체
    youtube= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, 
                   developerKey= DEVELOPER_KEY)
    
    # 검색결과 크롤링
    # 검색리스트를 부른다.
    # 쿼리에 명시된것과 일치한 결과물을 메소드에게 반환.
    search_response= youtube.search().list(
        q=query,
        part="snippet",
        maxResults=50
    ).execute()
    
    videos=[]
    channels=[]
    playlists=[]
    
    
    # 리스트에 적절한 각각의 결과물들을 추가한다.
    # 매칭된 비디오, 채널, 플레이리스트들을 리스트형태로 나타낸다.
    
    for search_result in search_response.get('items', []):
        if search_result['id']['kind']=='youtube#video':
            videos.append('%s (%s)' % (search_result['snippet']['title'],
                                       search_result['id']['videoId']))
            
        elif search_result['id']['kind']=='youtube#channel':
            channels.append('%s (%s)' %(search_result['snippet']['title'],
                                       search_result['id']['channelId']))
        elif search_result['id']['kind']=='youtube#playlist':
            playlists.append('%s (%s)' %(search_result['snippet']['title'],
                                        search_result['id']['playlistId']))
    return videos, channels, playlists

In [13]:

search_queries=['서울 강남 맛집', '서울 홍대 맛집', '서울 가성비 맛집',
               '서울 종로 맛집', '경기도 맛집', '서울 건국대 맛집']

videos, channels, playlists = youtube_search(search_queries[0])
videos

검색결과 =>  서울 강남 맛집


['[삼카슐랭] #10 서울 강남맛집 종합편 (with 쓰복만) (1tD2I9XO3-s)',
 '❤무한도전이 극찬한 강남의 맛있는집!!! 어육장 ㅣ강남역,역삼역 6분거리ㅣ강남맛집ㅣ하얀트리 (kP6uWsZedms)',
 '강남 맛집 여기다 있음 (W_NgtlZ1m0A)',
 '[한국오빠]믿고 가는 이영자 추천 서울 맛집 10곳 완벽정리 (배고픔주의) (Dav8BVt6nH8)',
 '011 향방 (서울 강남구 신사역) 중화요리 맛집 리뷰 &amp; 짬뽕 리뷰  a.k.a. 대통령들의 맛집 (CMlge7i-7kg)',
 '[강남 맛집] 강남인들만 안다는 중국집을 다녀와봤습니다/중국냉면 맛집 /in gangnam style Chinese dish (ZpXaNIa57uA)',
 '[돼동여지도]강남 맛집 (II_9DpUZOMQ)',
 '서울 최고의 부촌 강남구 완전 정복 (ft.서울여행,강남맛집)(korea seoul,travel seoul,travel korea) (3zhrGREtAA8)',
 '강남 노포 추억의 맛집 베스트 3 #08 (HUKBbChbWII)',
 '성공 100%를 보장하는 소개팅 맛집 대공개! (VMqjxLtilJQ)',
 '강남 맛집 탐방! [마루가메 제면, 은행골, 오코노미야끼 우와] 솔직 리뷰 먹방 (8ISoaFBC9GM)',
 '[맛집 소개] 강남역 맛집 TOP 10 (eEjn6wktrc4)',
 '논현역 맛집 &#39;회식&#39;/횟집추천/강남맛집추천/논현동맛집/서울맛집/강남모임장소/리뷰퀸 (4RdvT1set7c)',
 '강남에서 찾기힘든 한정식집 맛집발견! 70년 전통의 &#39;조양관&#39; 맛과 멋이있는 상견례명소 강남맛집투어 먹방 mukbnag [수아처럼] (Yem4xHWFtbU)',
 '[하슐랭로드 4화] 강남가면 이거 꼭 드세요!! 백종원 핫도그에 방송나온 떡볶이를 맛보다! (yQq_0ABbnhE)',
 '떡볶이 1위, 서울 분식 맛집 ( 떡볶이, 꼬치, 튀김, 순대 ) / 길거리음식 / 강남 맛집 (tOXiae5ORAE)',

In [14]:
videos, channels, playlists = youtube_search(search_queries[0])

검색결과 =>  서울 강남 맛집


In [15]:
videos

['[삼카슐랭] #10 서울 강남맛집 종합편 (with 쓰복만) (1tD2I9XO3-s)',
 '❤무한도전이 극찬한 강남의 맛있는집!!! 어육장 ㅣ강남역,역삼역 6분거리ㅣ강남맛집ㅣ하얀트리 (kP6uWsZedms)',
 '강남 맛집 여기다 있음 (W_NgtlZ1m0A)',
 '[한국오빠]믿고 가는 이영자 추천 서울 맛집 10곳 완벽정리 (배고픔주의) (Dav8BVt6nH8)',
 '서울 최고의 부촌 강남구 완전 정복 (ft.서울여행,강남맛집)(korea seoul,travel seoul,travel korea) (3zhrGREtAA8)',
 '[돼동여지도]강남 맛집 (II_9DpUZOMQ)',
 '[강남 맛집] 강남인들만 안다는 중국집을 다녀와봤습니다/중국냉면 맛집 /in gangnam style Chinese dish (ZpXaNIa57uA)',
 '떡볶이 1위, 서울 분식 맛집 ( 떡볶이, 꼬치, 튀김, 순대 ) / 길거리음식 / 강남 맛집 (tOXiae5ORAE)',
 '강남 노포 추억의 맛집 베스트 3 #08 (HUKBbChbWII)',
 '청담동에 있는 아시아 베스트 레스토랑 41위! 미슐랭에 갓성비까지 장착한 &quot;톡톡&quot;에 다녀왔습니다 (ja38rugu3eM)',
 '강남 맛집 탐방! [마루가메 제면, 은행골, 오코노미야끼 우와] 솔직 리뷰 먹방 (8ISoaFBC9GM)',
 '011 향방 (서울 강남구 신사역) 중화요리 맛집 리뷰 &amp; 짬뽕 리뷰  a.k.a. 대통령들의 맛집 (CMlge7i-7kg)',
 '성공 100%를 보장하는 소개팅 맛집 대공개! (VMqjxLtilJQ)',
 '[맛집 소개] 강남역 맛집 TOP 10 (eEjn6wktrc4)',
 '[하슐랭로드 4화] 강남가면 이거 꼭 드세요!! 백종원 핫도그에 방송나온 떡볶이를 맛보다! (yQq_0ABbnhE)',
 '[ 강남역맛집 ] 구구당 : - 강남에서 즐기는 홍콩감성  - 서울 맛집 감성 뿌개기#1 (fpHzsmC4h58)',
 '여기 숨겨뒀다가 드디어 촬영

In [16]:
channels

[]

In [17]:
playlists

['서울 _ 강남 맛집 (Restaurant recommendations for Gangnam, Seoul: South of Han River) (PLUjnnOepi47UgcRNuUpomQhj8Omc0J4wp)',
 '서울강남맛집 (PLFKeXaeIS5UQQHe2Fb971UMaYR8XgOfow)',
 '서울 강남맛집 (PLzXCi0hgJucgNJyUoka9xRPQoKQOVlhrd)',
 '[서울] 서울맛집을 찾아서 (PLfpYKKNa6ByHXSi7Py55dOUMNE2q-_fGg)']

> # 3. pageToken

In [22]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

# api key 객체 생성
DEVELOPER_KEY= "발급받은 유튜브 API키"
YOUTUBE_API_SERVICE_NAME="youtube"
YOUTUBE_API_VERSION="v3"

def youtube_search_with_pageToken(query):
    # build(googleapiclient.discovery) 객체 생성
    youtube= build(YOUTUBE_API_SERVICE_NAME,
                   YOUTUBE_API_VERSION,
                   developerKey= DEVELOPER_KEY)

    # 검색결과 크롤링
    search_response=youtube.search().list(
        q=query,
        pageToken='CDIQAA',
        order='date',
        part='snippet',
        maxResults=50
    ).execute()
    
    results=[]
    for search_result in search_response.get('items', []):
        if search_result['id']['kind']=='youtube#video':
            video_id=search_result['id']['videoId']
            title=search_result['snippet']['title']
            data={
                'url': 'https://youtube.com/watch?v='+video_id,
                'title':title
            }
            results.append(data)
    
    return results

In [23]:
search_responses=youtube_search_with_pageToken('강남역 맛집')
search_responses

[{'url': 'https://youtube.com/watch?v=ajiSSRpi5aI',
  'title': '[ VLOG ] 먹기만 하다 끝나는 브이로그🐷 / 일상브이로그ㅣ강남역맛집'},
 {'url': 'https://youtube.com/watch?v=-aJh8N_4mxE',
  'title': '[쑝vlog] 강남역 먹방 데이트 코스: 파스타 맛집 + 디저트 리뷰 vlog'},
 {'url': 'https://youtube.com/watch?v=QLoKRpWfit4',
  'title': '서울 강남 맛집 브이로그 | 돈까스, 수플레 팬케이크, 팟타이 | Yoonpic 윤픽'},
 {'url': 'https://youtube.com/watch?v=1tD2I9XO3-s',
  'title': '[삼카슐랭] #10 서울 강남맛집 종합편 (with 쓰복만)'},
 {'url': 'https://youtube.com/watch?v=SYpGj-pVRRg',
  'title': '강남역직장인 이 추천하는 강남역맛집 들름집, 쉑쉑버거'},
 {'url': 'https://youtube.com/watch?v=vPGTRbD6AwQ',
  'title': '강남역맛집 전통대구막창 막창뱅크'},
 {'url': 'https://youtube.com/watch?v=Is4NnCI36oY',
  'title': '강남역맛집 !! 더블트러블!!. 살찌고싶으신분들은여기로오세요 !!!'},
 {'url': 'https://youtube.com/watch?v=V66w7xWxTyk',
  'title': '강남역맛집/알부자/알찜/존맛탱집'},
 {'url': 'https://youtube.com/watch?v=H4beEM5pfcs',
  'title': '강남역맛집!!핫한돈까스집!! 정돈 !!! 드디어왔습니다 !!! 정돈 강남점'},
 {'url': 'https://youtube.com/watch?v=nFpAm1Jq3s8',
  'title': '강남 맛집 브이로그